In [1]:
import json
import pandas as pd
import numpy as np

from functions import infopld_normalize, word_normalize, preprocessing_submercado

## Loading data 

Neste projeto vamos explorar a relação entre consumo de energia e custo por submercado.

**Datasets** - https://www.ccee.org.br/portal/:
 - consumo.xlsx
 - infopld.xlsx (3 sheets)
 
**Informação auxiliar**:
 - estado2submercado.json (Mapeia estado - submercado)

In [2]:
consumo_df = pd.read_excel("data/consumo.xlsx")
infopld = pd.ExcelFile("data/infopld.xlsx")
infopld_2018_df = infopld.parse("PLD2018",encoding='ISO-8859-1') 
infopld_2019_df = infopld.parse("PLD2019") 
infopld_2020_df = infopld.parse("PLD2020") 

with open("data/estado2submercado.json", "r") as f:
    submercados = json.load(f)

## Cleaning 

In [3]:
# auxiliar info cleaning
estado2submercado = preprocessing_submercado(submercados)

#Consumo cleaning
estados_clean = consumo_df["Estado"].str.normalize('NFKD')\
                                    .str.encode('ascii', errors='ignore')\
                                    .str.decode('utf-8')\
                                    .str.strip()\
                                    .str.lower()

clean_data = pd.to_datetime(consumo_df['Data'], format='%d/%m/%Y', errors='raise')


consumo_df = consumo_df.assign(Estado = estados_clean)\
                       .assign(Data=clean_data)\
                       .fillna(method="ffill")\
                       .replace({"Estado":estado2submercado})


#pld cleaning
infopld18_clean = infopld_normalize(infopld_2018_df)
infopld19_clean = infopld_normalize(infopld_2019_df)
infopld20_clean = infopld_normalize(infopld_2020_df)

## Analysis 

###  Consumo semanal por submercado

In [4]:
infopld18_clean.head(10)

,submercado,semana,2018-01-01 00:00:00,2018-02-01 00:00:00,2018-03-01 00:00:00,2018-04-01 00:00:00,2018-05-01 00:00:00,2018-06-01 00:00:00,2018-07-01 00:00:00,2018-08-01 00:00:00,2018-09-01 00:00:00,2018-10-01 00:00:00,2018-11-01 00:00:00,2018-12-01 00:00:00
0,nordeste,1,182.690000,176.926667,189.380000,40.160000,154.51,380.750000,505.18,505.18,487.806667,386.736667,142.693333,59.180000
1,nordeste,2,179.980000,176.750000,223.400000,106.830000,154.51,321.060000,505.18,505.18,496.926667,326.100000,150.066667,66.410000
2,nordeste,3,162.106667,168.186667,236.266667,122.943333,165.95,463.783333,505.18,505.18,492.430000,270.663333,121.036667,79.246667
3,nordeste,4,190.690000,184.506667,218.666667,136.160000,201.88,478.433333,505.18,505.18,442.283333,229.960000,119.333333,87.423333
4,nordeste,5,176.926667,189.380000,229.530000,154.510000,380.75,505.180000,505.18,505.18,386.736667,142.693333,100.796667,53.860000
5,nordeste,6,0.000000,0.000000,40.160000,0.000000,0.00,505.180000,0.00,0.00,0.000000,0.000000,0.000000,0.000000
6,norte,1,168.486667,78.723333,40.160000,40.160000,40.16,380.750000,505.18,505.18,490.513333,386.736667,142.693333,59.073333
7,norte,2,165.653333,40.160000,40.160000,77.946667,40.16,321.060000,505.18,505.18,496.926667,326.100000,150.066667,66.156667
8,norte,3,142.390000,40.160000,40.160000,68.550000,115.01,463.783333,505.18,505.18,492.430000,270.663333,121.036667,40.160000
9,norte,4,145.796667,40.160000,40.160000,40.160000,201.88,478.433333,505.18,505.18,442.283333,229.960000,119.333333,40.160000


### Consumo vs preço 

In [5]:
columns_agg = infopld18_clean.filter(regex=r"\d{4}-\d{2}-\d{2}").columns
pd.pivot_table(infopld18_clean, values = columns_agg, index='submercado', aggfunc=[np.mean,np.std], fill_value=0).T

submercado                  nordeste       norte     sudeste         sul
mean 2018-01-01 00:00:00  148.732222  116.841667  150.866111  148.062222
     2018-02-01 00:00:00  149.291667   39.893889  156.578333  153.501111
     2018-03-01 00:00:00  189.567222   40.160000  191.878333  191.878333
     2018-04-01 00:00:00   93.433889   44.496111  100.423333  100.423333
     2018-05-01 00:00:00  176.266667  129.660000  266.565000  266.565000
     2018-06-01 00:00:00  442.397778  442.397778  468.448333  468.448333
     2018-07-01 00:00:00  420.983333  420.983333  420.983333  420.983333
     2018-08-01 00:00:00  420.983333  420.983333  420.983333  420.983333
     2018-09-01 00:00:00  384.363889  384.815000  384.363889  384.363889
     2018-10-01 00:00:00  226.025556  226.025556  226.025000  226.025000
     2018-11-01 00:00:00  105.654444  105.654444  105.653889  105.653889
     2018-12-01 00:00:00   57.686667   43.235000   71.608333   71.608333
std  2018-01-01 00:00:00   73.464394   65.796469   74.929622   74.613274
     2018-02-01 00:00:00   73.498091   24.897917   78.314581   78.393617
     2018-03-01 00:00:00   74.955786    0.000000   75.207266   75.207266
     2018-04-01 00:00:00   60.289425   27.327052   76.957230   76.957230
     2018-05-01 00:00:00  122.141355  142.343777  144.044322  144.044322
     2018-06-01 00:00:00   75.049881   75.049881   38.425913   38.425913
     2018-07-01 00:00:00  206.238871  206.238871  206.238871  206.238871
     2018-08-01 00:00:00  206.238871  206.238871  206.238871  206.238871
     2018-09-01 00:00:00  192.948160  193.241320  192.948160  192.948160
     2018-10-01 00:00:00  138.479441  138.479441  138.479842  138.479842
     2018-11-01 00:00:00   54.689138   54.689138   54.688687   54.688687
     2018-12-01 00:00:00   30.886322   23.568295   44.716392   44.716392